[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/monitor-sec-filings-python.ipynb)

# Monitor SEC Filings for Multiple Companies with Python -- Free, No API Key

Use **edgartools** to monitor SEC filings across a portfolio of companies in Python -- completely free, no API key or paid subscription required. Build a watchlist that tracks new filings as they appear on EDGAR.

**What you'll learn:**
- Get today's SEC filings in real time with `get_current_filings()`
- Build a watchlist to monitor specific companies
- Filter for material filings (10-K, 10-Q, 8-K, 13F)
- Detect earnings announcements and insider trades as they're filed

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get Today's SEC Filings in Real Time

The SEC updates its filing feed throughout the day. `get_current_filings()` gives you the latest filings as they appear:

In [ ]:
# Get filings filed today (or the most recent business day)
current = get_current_filings()
print(f"Current filings: {len(current)}")
current

## Filter Current Filings by Form Type

Focus on material filings that move markets:

In [ ]:
# Filter for key filing types
earnings = current.filter(form="8-K")
print(f"8-K filings (may include earnings): {len(earnings)}")

insider_trades = current.filter(form="4")
print(f"Form 4 (insider trades): {len(insider_trades)}")

annual_reports = current.filter(form="10-K")
print(f"10-K (annual reports): {len(annual_reports)}")

quarterly_reports = current.filter(form="10-Q")
print(f"10-Q (quarterly reports): {len(quarterly_reports)}")

## Build a Company Watchlist

Monitor a portfolio of companies for new filings. Define your watchlist and scan recent filings:

In [ ]:
# Define your watchlist
watchlist = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "TSLA", "META", "JPM", "V", "JNJ"]

# Get CIKs for the watchlist (for filtering)
watchlist_ciks = []
for ticker in watchlist:
    c = Company(ticker)
    watchlist_ciks.append(c.cik)
    print(f"  {ticker:6s} CIK: {c.cik}  {c.name}")

In [ ]:
# Check current filings against the watchlist
current = get_current_filings()
watchlist_filings = current.filter(cik=watchlist_ciks)

if len(watchlist_filings) > 0:
    print(f"Found {len(watchlist_filings)} filings from your watchlist:")
    watchlist_filings
else:
    print("No filings from your watchlist today (yet).")
    print("Try running this during market hours for more activity.")

## Monitor Recent Filings for a Company

Check what a specific company has filed recently:

In [ ]:
# See NVIDIA's recent filings across all form types
nvda = Company("NVDA")
recent = nvda.get_filings().head(15)

print(f"Last 15 filings for {nvda.name}:")
for f in recent:
    print(f"  {f.filing_date}  {f.form:10s}  {f.primary_doc_description or ''}")

## Detect Earnings Announcements

8-K filings with Item 2.02 contain earnings releases. Scan for them across your watchlist:

In [ ]:
# Scan for recent earnings from watchlist companies
print("Scanning for recent earnings announcements...\n")

for ticker in ["AAPL", "MSFT", "NVDA", "GOOG"]:
    c = Company(ticker)
    eightks = c.get_filings(form="8-K").head(5)
    
    for f in eightks:
        try:
            ek = f.obj()
            if ek.has_earnings:
                print(f"  {ticker:6s} {f.filing_date}  EARNINGS  Items: {ek.items}")
                break
        except Exception:
            pass

## Track Insider Trading Across the Portfolio

Monitor Form 4 filings to detect insider buying and selling:

In [ ]:
# Check recent insider trades for a few companies
print("Recent insider trades:\n")

for ticker in ["AAPL", "TSLA", "NVDA"]:
    c = Company(ticker)
    form4s = c.get_filings(form="4").head(3)
    
    for f in form4s:
        try:
            summary = f.obj().get_ownership_summary()
            direction = "BUY" if summary.net_change > 0 else "SELL"
            value_str = f"${abs(summary.net_value):,.0f}" if summary.net_value else "N/A"
            print(f"  {ticker:6s} {f.filing_date}  {direction:4s}  {summary.insider_name:30s}  {value_str}")
        except Exception:
            pass

## Build a Filing Summary Dashboard

Combine everything into a watchlist status report:

In [ ]:
import pandas as pd

# Build a summary of recent filing activity
summary_data = []

for ticker in ["AAPL", "MSFT", "GOOG", "NVDA", "TSLA"]:
    c = Company(ticker)
    filings = c.get_filings()
    
    # Count recent filings by type
    recent_30 = filings.filter(filing_date="2025-01-01:")  # Recent filings
    
    summary_data.append({
        "Ticker": ticker,
        "Company": c.name,
        "Total Filings": len(recent_30),
        "Latest Filing": filings[0].filing_date if len(filings) > 0 else "N/A",
        "Latest Form": filings[0].form if len(filings) > 0 else "N/A",
    })

pd.DataFrame(summary_data)

## Historical Filing Search by Date

Use `get_filings()` with date parameters to search any historical period:

In [ ]:
# Get all 10-K filings from a specific date range
filings = get_filings(form="10-K", filing_date="2025-01-01:2025-03-31")
print(f"10-K filings in Q1 2025: {len(filings)}")
filings.head(10)

## Why EdgarTools?

EdgarTools is free and open-source. Compare monitoring SEC filings:

**With edgartools (free, no API key):**
```python
current = get_current_filings()
earnings = current.filter(form="8-K")
# Done -- real-time filings, filtered and ready
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import RealTimeApi
api = RealTimeApi(api_key="YOUR_PAID_API_KEY")
filings = api.get_filings({"formType": "8-K"})  # Costs per request
```

With edgartools, real-time filing monitoring is built in -- no webhooks, no API key, no monthly fee.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# Real-time filings
current = get_current_filings()          # Today's filings
current.filter(form="8-K")              # Filter by form type
current.filter(cik=[1318605, 320193])   # Filter by company CIK

# Company-specific monitoring
filings = Company("AAPL").get_filings()  # All filings
filings.filter(form="10-K")             # Annual reports only
filings.filter(filing_date="2025-01-01:")  # Date range

# Historical search
get_filings(form="10-K", filing_date="2025-01-01:2025-03-31")
```

## What's Next

You've learned how to monitor SEC filings for multiple companies with Python. Here are related tutorials:

- [Get Today's SEC Filings with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-filings-today-python.ipynb)
- [Extract 8-K Earnings Releases with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/8k-earnings-release-python.ipynb)
- [Track Insider Trading from SEC Form 4](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/insider-trading-sec-form4-python.ipynb)
- [Search SEC Filings with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/search-sec-filings-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*